In [1]:
# STEP 1: Install Dependencies

!pip install -U spacy pandas scikit-learn tqdm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 134.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en

In [2]:
# STEP 2: Import Libraries

import pandas as pd
import spacy
from spacy.training import Example
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
# STEP 3: Load Dataset

df = pd.read_csv("NER_dataset.csv", encoding='latin1')

# Expected columns:
# Sentence | Entity | Label


In [4]:
# STEP 4: Convert to spaCy Format

training_data = []
current_sentence_words = []
current_sentence_tags = []

# Using tqdm to show progress
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing dataset to spaCy format"):
    word = str(row['Word'])
    ner_tag = str(row['Tag'])  # e.g., 'O', 'B-PER', 'I-PER'

    # Check if a new sentence starts
    if pd.notna(row['Sentence #']):
        # Process the previous sentence if it exists
        if current_sentence_words:
            full_text = " ".join(current_sentence_words)
            entities = []
            current_entity_start_char = -1
            current_entity_label = None
            word_start_char = 0  # Tracks character offset in full_text

            for word_idx, (w, tag) in enumerate(zip(current_sentence_words, current_sentence_tags)):
                if tag.startswith('B-'):
                    # If an entity was ongoing, close it before starting a new one
                    if current_entity_start_char != -1:
                        entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
                    current_entity_start_char = word_start_char
                    current_entity_label = tag[2:]  # Extract label, e.g., 'PER' from 'B-PER'
                elif tag.startswith('I-'):
                    # Continue entity, but handle potential errors in IOB sequencing
                    # If no B-tag preceded or label mismatch, treat as new B-tag for robustness
                    if current_entity_start_char == -1 or current_entity_label != tag[2:]:
                        if current_entity_start_char != -1:
                            entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
                        current_entity_start_char = word_start_char
                        current_entity_label = tag[2:]
                elif tag == 'O':
                    if current_entity_start_char != -1:  # Entity just ended
                        entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
                    current_entity_start_char = -1  # Reset
                    current_entity_label = None

                word_start_char += len(w) + 1  # +1 for the space after the word

            # After processing all words in a sentence, check if an entity was still ongoing
            if current_entity_start_char != -1:
                entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))

            training_data.append((full_text, {"entities": entities}))

        # Reset for the new sentence
        current_sentence_words = [word]
        current_sentence_tags = [ner_tag]
    else:
        # Continue adding words and tags to the current sentence
        current_sentence_words.append(word)
        current_sentence_tags.append(ner_tag)

# After the loop, process the very last sentence
if current_sentence_words:
    full_text = " ".join(current_sentence_words)
    entities = []
    current_entity_start_char = -1
    current_entity_label = None
    word_start_char = 0

    for word_idx, (w, tag) in enumerate(zip(current_sentence_words, current_sentence_tags)):
        if tag.startswith('B-'):
            if current_entity_start_char != -1:
                entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
            current_entity_start_char = word_start_char
            current_entity_label = tag[2:]
        elif tag.startswith('I-'):
            if current_entity_start_char == -1 or current_entity_label != tag[2:]:
                if current_entity_start_char != -1:
                    entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
                current_entity_start_char = word_start_char
                current_entity_label = tag[2:]
        elif tag == 'O':
            if current_entity_start_char != -1:
                entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))
            current_entity_start_char = -1
            current_entity_label = None
        word_start_char += len(w) + 1

    if current_entity_start_char != -1:
        entities.append((current_entity_start_char, word_start_char - 1, current_entity_label))

    training_data.append((full_text, {"entities": entities}))


Processing dataset to spaCy format: 100%|██████████| 2307/2307 [00:00<00:00, 28834.43it/s]


In [5]:
# STEP 5: Train-Test Split

train_data, test_data = train_test_split(
    training_data, test_size=0.2, random_state=42
)

In [6]:
# STEP 6: Load spaCy Model

nlp = spacy.load("en_core_web_sm")
ner = nlp.get_pipe("ner")

In [7]:
# STEP 7: Add Labels

for _, annotations in train_data:
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

In [8]:
# STEP 8: Train Model

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()

    for epoch in range(20):
        losses = {}
        for text, annotations in tqdm(train_data):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
        print(f"Epoch {epoch+1}, Loss:", losses)

100%|██████████| 80/80 [00:01<00:00, 47.50it/s]


Epoch 1, Loss: {'ner': np.float32(286.62292)}


100%|██████████| 80/80 [00:01<00:00, 48.91it/s]


Epoch 2, Loss: {'ner': np.float32(187.12917)}


100%|██████████| 80/80 [00:01<00:00, 50.48it/s]


...

Epoch 18, Loss: {'ner': np.float32(41.544777)}


100%|██████████| 80/80 [00:01<00:00, 50.62it/s]


Epoch 19, Loss: {'ner': np.float32(29.973618)}


100%|██████████| 80/80 [00:01<00:00, 50.68it/s]

Epoch 20, Loss: {'ner': np.float32(20.888296)}


In [9]:
# STEP 9: Save Model

nlp.to_disk("/content/ner_model")

In [10]:
# STEP 10: Load Model for Testing

nlp_test = spacy.load("/content/ner_model")

In [11]:
# STEP 11: Test on New Sentence

text = "Microsoft was founded by Bill Gates in USA"
doc = nlp_test(text)

print("\nPredicted Entities:")
for ent in doc.ents:
    print(ent.text, "->", ent.label_)


Predicted Entities:
Microsoft -> org
Bill Gates in USA -> org


In [13]:
# STEP 12: Evaluate on Test Data

correct = 0
total = 0

for text, annotations in test_data:
    doc = nlp_test(text)
    predicted = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    actual = annotations["entities"]

    for ent in actual:
        total += 1
        if ent in predicted:
            correct += 1

accuracy = correct / total if total > 0 else 0
print("\nTest Accuracy:", accuracy)


Test Accuracy: 0.8888888888888888
